# A CNN model based on pytorch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as functional
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as py
import time

##%matplotlib inline

input_size = 28
num_classes = 10
num_epochs = 3
batch_size = 64

## Make train datasets and test datasets
train_dataset = datasets.MNIST(root='./DATA', train=True, transform=transforms.ToTensor(), download=True)

test_dataset = datasets.MNIST(root='DATA', train=False, transform=transforms.ToTensor())

## Make batch datasets
train_load = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_load = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

## Create CNN structure
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(1, 16, 5, 1, 2), nn.ReLU(), nn.MaxPool2d(2))

        self.conv2 = nn.Sequential(nn.Conv2d(16, 32, 5, 1, 2), nn.ReLU(), nn.MaxPool2d(2))

        ##self.conv3 = nn.Sequential(nn.Conv2d(32, 64, 5, 1, 2), nn.ReLU(), nn.MaxPool2d(2))

        self.out = nn.Linear(32 * 7 * 7, 10)
        ##self.out = nn.Linear(64 * 3 * 3, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        ##x = self.conv3(x)
        x = x.view(x.size(0), -1)
        output = self.out(x)
        return output


def accuracy(predictions, labels):
    pred = torch.max(predictions.data, 1)[1]
    rights = pred.eq(labels.data.view_as(pred)).sum()
    return rights, len(labels)


net = CNN()

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(net.parameters(), lr=0.001)

time_begin = time.time()

for epoch in range(num_epochs):
    train_rights = []

    for batch_idx, (data, target) in enumerate(train_load):
        net.train()
        output = net(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        right = accuracy(output, target)
        train_rights.append(right)

        if batch_idx % 100 == 0:
            net.eval()
            val_rights = []

            for (data, target) in test_load:
                output = net(data)
                right = accuracy(output, target)
                val_rights.append(right)
            train_r = (sum([tup[0] for tup in train_rights]), sum([tup[1] for tup in train_rights]))
            val_r = (sum([tup[0] for tup in val_rights]), sum([tup[1] for tup in val_rights]))

            print('epoch: {} [{}/{} ({:.0f}%)]\t loss: {:.6f}\t train accuracy: {:.2f}%\t right rate: {:.2f}%'.format(
                epoch, batch_idx * batch_size, len(train_load.dataset),
                       100. * batch_idx / len(train_load),
                loss.data,
                       100. * train_r[0].numpy() / train_r[1],
                       100. * val_r[0].numpy() / val_r[1]))
time_end = time.time()

print(time_end - time_begin,'s')